## Import

In [0]:
pip install -U ipykernel

In [0]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

## Data Load

In [0]:
apply_train_df = pd.read_csv('apply_train.csv')

## User-Item Matrix / Similarity / Score

In [0]:
# 사용자-아이템 행렬 생성: 구직자가 해당 채용 공고에 지원했으면 1, 아니면 0으로 설정
user_item_matrix = apply_train_df.groupby(['resume_seq', 'recruitment_seq']).size().unstack(fill_value=0)
user_item_matrix[user_item_matrix > 1] = 1
print(user_item_matrix)

In [0]:
# 사용자 간의 유사성 계산
user_similarity = cosine_similarity(user_item_matrix)
print(user_similarity)

In [0]:
np.abs(user_similarity)

In [0]:
np.array([np.abs(user_similarity).sum(axis=1)]).T

In [0]:
# 추천 점수 계산
user_predicted_scores = user_similarity.dot(user_item_matrix) / np.array([np.abs(user_similarity).sum(axis=1)]).T
print(user_predicted_scores)

## Prediction

In [0]:
user_item_matrix.index

In [0]:
for idx, user in enumerate(user_item_matrix.index):
    print(idx, user)

In [0]:
# 이미 지원한 채용 공고 제외하고 추천
recommendations = []
for idx, user in enumerate(user_item_matrix.index):
    # 해당 사용자가 지원한 채용 공고
    applied_jobs = set(user_item_matrix.loc[user][user_item_matrix.loc[user] == 1].index)
    
    # 해당 사용자의 추천 점수 (높은 점수부터 정렬)
    sorted_job_indices = user_predicted_scores[idx].argsort()[::-1] # argsort(): 원본 행렬을 sorting 하고 원래 인덱스의 값을 반환
    print(sorted_job_indices)

In [0]:
# 이미 지원한 채용 공고 제외하고 추천
recommendations = []
for idx, user in enumerate(user_item_matrix.index):
    # 해당 사용자가 지원한 채용 공고
    applied_jobs = set(user_item_matrix.loc[user][user_item_matrix.loc[user] == 1].index)
    
    # 해당 사용자의 추천 점수 (높은 점수부터 정렬)
    sorted_job_indices = user_predicted_scores[idx].argsort()[::-1]
    recommended_jobs = [job for job in user_item_matrix.columns[sorted_job_indices] if job not in applied_jobs][:5]
    
    for job in recommended_jobs:
        recommendations.append([user, job])
print(recommendations[0])

## Submission

In [0]:
# sample_submission.csv 형태로 DataFrame 생성
top_recommendations = pd.DataFrame(recommendations, columns=['resume_seq', 'recruitment_seq'])

top_recommendations.to_csv('./baseline_submit.csv', index=False)